# Homework # 6: Spark DataFrames Exercise

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Homework_6").getOrCreate()

#### Load the Walmart Stock CSV File from URL (https://raw.githubusercontent.com/oakabc/DEA/refs/heads/main/6%20-%20Aggregating%20and%20Summarizing%20Data/walmart_stock.csv).

In [ ]:
import os
import requests

url = "https://raw.githubusercontent.com/oakabc/DEA/refs/heads/main/6%20-%20Aggregating%20and%20Summarizing%20Data/walmart_stock.csv"
local_file = "walmart_stock.csv"

response = requests.get(url)
if response.status_code == 200:
   with open(local_file, "wb") as f:
       f.write(response.content)

#### What are the column names?

In [ ]:
df = spark.read.csv(local_file, header=True, inferSchema=True)
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [ ]:
from pyspark.sql.functions import col
df = df.withColumn("Date", col("Date").cast("timestamp"))
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [ ]:
for row in df.head(5):
  print(row)
  print('\n')

Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)


Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)


Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)


Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)


Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)




#### Use describe() to learn about the DataFrame.

In [ ]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.cast.html)

In [ ]:
df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [ ]:
from pyspark.sql.functions import format_number, col, regexp_replace
describe_df = df.describe()
formatted_describe_df = describe_df.select(
    col("summary"),
    format_number(col("Open").cast("float"), 2).alias("Open"),
    format_number(col("High").cast("float"), 2).alias("High"),
    format_number(col("Low").cast("float"), 2).alias("Low"),
    format_number(col("Close").cast("float"), 2).alias("Close"),
    regexp_replace(format_number(col("Volume").cast("double"), 0), ",", "").alias("Volume")
)

In [ ]:
formatted_describe_df.show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519781|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [ ]:
df = df.withColumn("HV Ratio", col("High") / col("Volume"))
df.select(col("HV Ratio")).show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



#### What day had the Peak High in Price?

In [ ]:
from pyspark.sql.functions import max

max_high = df.agg(max("High")).collect()[0][0]
result_df = df.filter(col("High") == max_high).select("Date")

result_df.collect()[0]

Row(Date=datetime.datetime(2015, 1, 13, 0, 0))

#### What is the mean of the Close column?

In [ ]:
from pyspark.sql.functions import mean
mean_close = df.agg(mean("Close"))

mean_close.show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [ ]:
from pyspark.sql.functions import min

In [ ]:
min_max = df.agg(
    max("Volume").alias("max(Volume)"),
    min("Volume").alias("min(Volume)")
)

min_max.show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [ ]:
df.filter(col("Close") < 60).count()

81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [ ]:
M_80 = df.filter(col("High") > 80).count()
percentage = (M_80 / df.count()) * 100
print(percentage)

9.141494435612083


#### What is the Pearson correlation between High and Volume?
#### [Hint](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.stat.Correlation.html)

In [ ]:
from pyspark.sql import Row
from pyspark.sql.functions import lit

correlation = df.stat.corr("High", "Volume")
result_df = spark.createDataFrame([Row(**{"corr(High, Volume)": correlation})])

result_df.show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### What is the max High per year?

In [ ]:
from pyspark.sql.functions import year
df = df.withColumn("Year", year(df["Date"]))

group_data = df.groupBy("Year").agg({"High" : "max"})
group_data.show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months.

In [ ]:
from pyspark.sql.functions import month
df = df.withColumn("Month", month(df["Date"]))

group_data = df.groupBy("Month").agg({"Close" : "avg"})
group_data.orderBy("Month").show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+



# Merry Christmas and Happy New Year !